In [2]:
import os
import json
from tqdm.auto import tqdm
from time import sleep
from typing import List, Any, Union


def load_jsonl(fpath: str) -> List[Union[dict, list]]:
    with open(fpath, "r") as f:
        data = f.readlines()
    data = [json.loads(x) for x in data]
    return data


load koalpaca 1.0 data


In [8]:
with open("./alpaca_data.json", "r") as f:
    alpaca_data = json.load(f)

In [10]:
len(alpaca_data)


101622

load koalpaca 1.1 data (naver kin)


In [12]:
alpaca_data2 = load_jsonl("./KoAlpaca_v1.1.jsonl")

In [13]:
len(alpaca_data2)


21155

load kochatgpt sft data


In [14]:
with open("../ref/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl", "r") as f:
    kochat_data = json.load(f)

In [15]:
len(kochat_data)


12000

---

- 데이터 형태 통일
- RM 용 데이터 스플릿 및 인덱스 저장 해 두기
- 새 SFT 데이터 = kochat-sft + koalpaca-1.0 20% + koalpaca-1.1 50%


In [20]:
from pprint import pprint

pprint(
    (alpaca_data[0],
    alpaca_data2[0],
    kochat_data[0],)
)

({'input': '',
  'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
  'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'},
 {'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
  'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n'
            '\n'
            '식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 '
            '부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 '
            '되고, 고구마는 식물의 뿌리 부분입니다.\n'
            '\n'
            ' 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n'
            '\n'
            '고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 '
            '활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
  'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'},
 {'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 "
                '고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 '
                '알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니

In [29]:
# make sampling index
import random
random.seed(777)

In [31]:
alpaca_index = random.sample(range(len(alpaca_data)), int(len(alpaca_data)*0.2))
alpaca2_index = random.sample(range(len(alpaca_data2)), int(len(alpaca_data2)*0.5))

In [49]:
# save index
with open('./new/sft-index.json', 'w') as f:
    idxs = {
        'koalpaca-1.0-sft': alpaca_index,
        'koalpaca-1.1-sft': alpaca2_index,
    }

    json.dump(idxs, f, ensure_ascii=False, indent=2)

In [34]:
len(alpaca_index), len(alpaca2_index)

(20324, 10577)

In [52]:
# merge all
alpaca_smp = []
for i in alpaca_index:
    x = alpaca_data[i]
    alpaca_smp.append(x)

alpaca2_smp = []
for i in alpaca2_index:
    x = alpaca_data2[i]
    alpaca2_smp.append(
        {"instruction": x['instruction'], "input": "", 'output': x['output']}
    )

kochat_smp = [
    {"instruction": x['prompt'], "input": "", 'output': x['completion']}
    for x in kochat_data
]

In [53]:
pprint(
    (alpaca_smp[0],
    alpaca2_smp[0],
    kochat_smp[0],)
)

({'input': '버락 오바마',
  'instruction': 'GPT를 사용하여 정치인의 캐리커처를 생성합니다.',
  'output': '버락 오바마의 캐리커처는 미소가 있는 마주보는 얼굴, 비싼 정장, 미국 국기가 그려진 벽지와 같은 청사진을 갖출 '
            '것입니다.'},
 {'input': '',
  'instruction': '일식집을 창업하려면 어떻게 기술을 익혀야 할까요? 부산에서 유명한 일식집은 어디인가요?\n'
                 '저는 3년 내에 일식집 창업을 계획하고 있습니다. 하지만 요리 실력을 어떻게 쌓아야 할지 모르겠습니다. 직접 '
                 '일식집에 들어가서 일을 배우는 것이 좋을까요? 또, 부산에서 유명한 일식집이 어디 있는지 알려주세요.',
  'output': '일식집을 창업하실 계획이신다면 3년 안에 일식집 창업은 어려울 것입니다. 일식집은 다른 음식점에 비해 전문적인 실력이 '
            '요구되기 때문입니다. 따라서, 돈이 있다면 주방장을 구해서 창업하는 것이 좋습니다. 만약 직접 일식집에서 일을 하면서 '
            '실력을 쌓으려 한다면 먼저 조리기능사 자격증을 필수적으로 따야 합니다. 학원에 등록하여 자격증을 따신 후, 어디든 '
            '들어가서 밑바닥부터 착실히 배워야 합니다. 하지만 이러한 과정은 쉽지 않을 것입니다. \n'
            '또한, 부산에서 유명한 일식집은 여러 곳이 있지만, 예전에 방영된 TV 프로그램이나 지인의 추천 등으로 유명해진 '
            '곳들이 있으니 이러한 곳을 참고하시면 될 것입니다. \n'
            '그러나, 일식집을 창업하시려면 먼저 횟집이나 초밥 전문점 등을 시작하시는 것이 좋습니다. 3년 안에 일식집 창업이 '
            '어렵다면, 처음부터 일식집을 거창하게 시작하는 것보다는 작은 규모의 레스토랑부터 시작해 경험을 쌓는 것이 유리합니다. 

In [54]:
new_sft_data = alpaca_smp + alpaca2_smp + kochat_smp
random.shuffle(new_sft_data)

In [55]:
len(new_sft_data)

42901

In [56]:
# save
with open('./new/sft.json', 'w') as f:
    json.dump(new_sft_data, f, ensure_ascii=False, indent=2)